In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
pwd

'/content'

In [5]:
cd drive/MyDrive/tsai_yolo/YoloV3-master

/content/drive/MyDrive/tsai_yolo/YoloV3-master


In [29]:
!python train.py --data data/dataset/custom.data --batch 10 --cache --epochs 25 --nosave


Namespace(epochs=25, batch_size=10, accumulate=4, cfg='cfg/yolov3-spp.cfg', data='data/dataset/custom.data', multi_scale=False, img_size=[512], rect=False, resume=False, nosave=True, notest=False, evolve=False, bucket='', cache_images=True, weights='weights/yolov3-spp-ultralytics.pt', name='', device='', adam=False, single_cls=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15102MB)

2024-04-19 19:07:17.891305: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 19:07:17.891365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 19:07:17.892880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for

In [9]:
pwd


'/content/drive/MyDrive/tsai_yolo/YoloV3-master'

In [33]:
import cv2
import torch
from models import Darknet
from utils.utils import non_max_suppression, scale_coords
from utils.datasets import letterbox
from PIL import Image

def load_model(cfg_path, weights_path):
    # Load YOLOv3 model architecture
    model = Darknet(cfg_path)
    # Load weights
    model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu'))['model'])
    model.eval()
    return model

def detect_objects(image_path, model, conf_thres=0.3, iou_thres=0.6, img_size=416):
    img = cv2.imread(image_path)
    # Resize image using letterbox resize
    img = letterbox(img, img_size, 416)[0]
    # Convert image to RGB and then to a torch tensor
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = torch.from_numpy(img.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)
    # Perform inference
    with torch.no_grad():
        detections = model(img)
        detections = non_max_suppression(detections, conf_thres, iou_thres)
    return detections[0]

def draw_boxes(image_path, detections, class_names):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if detections is not None:
        # Rescale boxes to original image size
        detections = scale_coords(detections, 416, img.shape[:2])
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
            box_w = x2 - x1
            box_h = y2 - y1
            color = (0, 255, 0)  # Green color for bounding box
            label = f'{class_names[int(cls_pred)]}: {conf:.2f}'
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    # Convert image back to BGR for displaying with OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

# Path to your custom YOLOv3 config file
cfg_path = '/content/drive/MyDrive/tsai_yolo/YoloV3-master/cfg/yolov3-custom.cfg'
# Path to your custom YOLOv3 weights file
weights_path = '/content/drive/MyDrive/tsai_yolo/YoloV3-master/weights/last.pt'
# Path to the image you want to detect objects in
image_path = r'data/dataset/images/0025.jpg'
# Path to your custom class names file
class_names_path = 'data/dataset/custom.names'

# Load the model
model = load_model(cfg_path, weights_path)

# Load class names
class_names = []
with open(class_names_path, 'r') as file:
    class_names = file.read().splitlines()

# Detect objects in the image
detections = detect_objects(image_path, model)

# Draw bounding boxes and labels on the image
output_image = draw_boxes(image_path, detections, class_names)

# Display the output image
# cv2.imshow('Object Detection', output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Model Summary: 225 layers, 6.29987e+07 parameters, 6.29987e+07 gradients


RuntimeError: shape '[1, 3, 6, 13, 13]' is invalid for input of size 43095

In [27]:
from google.colab.patches import cv2_imshow
cv2_imshow(output_image)

NameError: name 'output_image' is not defined

In [31]:
!python detect.py --cfg cfg/yolov3-custom.cfg --source data/dataset/test --output /content/drive/MyDrive/tsai_yolo/YoloV3-master/output --weights weights/last.pt


Namespace(cfg='cfg/yolov3-custom.cfg', names='data/customdata/custom.names', weights='weights/last.pt', source='data/dataset/test', output='/content/drive/MyDrive/tsai_yolo/YoloV3-master/output', img_size=512, conf_thres=0.3, iou_thres=0.6, fourcc='mp4v', half=False, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15102MB)

Model Summary: 225 layers, 6.29987e+07 parameters, 6.29987e+07 gradients
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Traceback (most recent call last):
  File "/content/drive/MyDrive/tsai_yolo/YoloV3-master/detect.py", line 186, in <module>
    detect()
  File "/content/drive/MyDrive/tsai_y

In [36]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

from models import *
from utils.utils import *

# Define the function for object detection
def detect(model, img_path, img_size=512, conf_thres=0.3, iou_thres=0.6, device=''):
    # Load image
    img0 = cv2.imread(img_path)  # BGR
    img = letterbox(img0, new_shape=img_size)[0]

    # Normalize image
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, HWC to CHW
    img = np.ascontiguousarray(img)

    if torch.cuda.is_available():
      device = torch.device('cuda')
    else:
      device = torch.device('cpu')

    # Convert image to tensor
    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0

    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if det is not None and len(det):
            # Rescale boxes from 416 to image size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            # Draw bounding boxes and labels
            for *xyxy, conf, cls in det:
                label = f'{names[int(cls)]} {conf:.2f}'
                plot_one_box(xyxy, img0, label=label, color=colors[int(cls)])

    return img0

# Path to your custom YOLOv3 weights file
weights_path = '/content/drive/MyDrive/tsai_yolo/YoloV3-master/weights/last.pt'
# Path to the image you want to detect objects in
image_path = r'data/dataset/images/0025.jpg'

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Darknet(cfg='cfg/yolov3-custom.cfg', img_size=512)
model.load_state_dict(torch.load(weights_path, map_location=device)['model'])
model.to(device).eval()

# Run detection
output_image = detect(model, image_path)

# Display the output image
cv2_imshow(output_image)


Model Summary: 225 layers, 6.29987e+07 parameters, 6.29987e+07 gradients


RuntimeError: shape '[1, 3, 6, 16, 16]' is invalid for input of size 65280